<div style="display: table; width: 100%;">
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 70%;">
    <h2>Maestría en Ciencia de Datos y Máquinas de Aprendizaje</h2>
    <h1>Inteligencia Artificial: Data Mining I - Tarea 2</h1>
  </div>
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 30%;">
    <img src="https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea3/blob/main/Assets/UideLogo.png?raw=true" alt="Texto alternativo" style="width:50%;">
  </div>
</div>
<hr />

#  Clustering
##  Breast Cancer Wisconsin Diagnostic Dataset

### 🟦 **Autores - Estudiantes - Grupo 7:**

&nbsp;&nbsp;&nbsp;&nbsp;💻 Luis Miguel Ramírez      
&nbsp;&nbsp;&nbsp;&nbsp;💻 Aviles Paute José     
&nbsp;&nbsp;&nbsp;&nbsp;💻 Espinoza Bone José    

### 🟦 Código fuente
[https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea3](https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea3)


**Fecha:** 11 de octubre de 2025

### 🟦 Introducción 
Breast Cancer Wisconsin Diagnostic Dataset es un conjunto de datos ampliamente utilizado en proyectos
de ciencia de datos y aprendizaje automático, especialmente para prácticas de clasificación y agrupamiento.

Fue recopilado por el Dr. William H. Wolberg en la Universidad de Wisconsin, y contiene información sobre
características de células tumorales obtenidas a partir de imágenes digitales de biopsias de tejido mamario.

El objetivo de estos datos es analizar las características físicas de los núcleos celulares para ayudar
a distinguir entre tumores malignos y benignos.

### 🟦 Objetivo General    
Aplicar técnicas de regresión, clasificación y redes neuronales para explorar y analizar 
patrones en los datos del cáncer de mama, con el fin de: 
1. Identificar relaciones entre variables mediante regresión lineal y logística. 
2. Evaluar el desempeño de un modelo de red neuronal para clasificación. 
3. Comparar resultados y probabilidades predichas por cada modelo. 

### 🟦 Objetivos Específicos      
- Realizar análisis exploratorio del dataset. 
- Implementar regresión lineal multivariable para predecir características continuas. 
- Implementar regresión logística para clasificar tumores malignos y benignos. 
- Diseñar y entrenar un MLP en PyTorch para clasificación binaria. 
- Visualizar resultados y comparar desempeño entre métodos. 



## 0️⃣ Preparar entorno (Funciones utilitarias, instalar libs)

In [ ]:
import sys
import subprocess
import os
from pathlib import Path
from enum import Enum
import zipfile
from typing import Optional, Iterable
from dataclasses import dataclass
from typing import cast
from typing import Tuple
from types import SimpleNamespace

# Libs a instalar
LIBS = [
    "matplotlib",
    "numpy",
    "pandas",
    "seaborn",
    "scikit-learn",
    "requests",
    "wcwidth",
]

class ConsoleColor(Enum):
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"
    RESET = "\033[0m"


def PrintColor(message: str, color: ConsoleColor) -> str:
    RESET = ConsoleColor.RESET.value
    return f"{color.value}{message}{RESET}"


def ShowMessage(
    message: str, title: str, icon: str, color: ConsoleColor, end: str = "\n"
):
    colored_title = PrintColor(icon + f"  " + title.upper() + ":", color)
    print(f"{colored_title} {message}", end=end)


def ShowInfoMessage(
    message: str, title: str = "Info", icon: str = "ℹ️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.CYAN, end)


def ShowSuccessMessage(
    message: str, title: str = "Success", icon: str = "✅", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.GREEN, end)


def ShowErrorMessage(
    message: str, title: str = "Error", icon: str = "❌", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.RED, end)


def ShowWarningMessage(
    message: str, title: str = "Warning", icon: str = "⚠️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.YELLOW, end)


# Funcion para ejecutar comandos
def RunCommand(
    commandList: list[str], printCommand: bool = True, printError: bool = True
) -> subprocess.CompletedProcess[str]:
    print("⏳", " ".join(commandList))

    if printCommand:
        proc = subprocess.Popen(
            commandList,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True,
        )

        out_lines: list[str] = []
        assert proc.stdout is not None
        for line in proc.stdout:
            print(line, end="")
            out_lines.append(line)

        proc.wait()
        err_text = ""
        if proc.stderr is not None:
            err_text = proc.stderr.read() or ""

        if proc.returncode != 0 and printError and err_text:
            ShowErrorMessage(err_text, "", end="")
            # print(err_text, end="")

        return subprocess.CompletedProcess(
            args=commandList,
            returncode=proc.returncode,
            stdout="".join(out_lines),
            stderr=err_text,
        )

    else:
        result = subprocess.run(
            commandList, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        if result.returncode != 0 and printError and result.stderr:
            ShowErrorMessage(result.stderr, "", end="")
            # print(result.stderr, end="")
        return result


# Función para instalar las dependencias
def InstallDeps(libs: Optional[list[str]] = None):
    print("ℹ️ Installing deps.")
    printCommand = False
    printError = True
    RunCommand(
        [sys.executable, "-m", "pip", "install", "--upgrade", "pip"],
        printCommand=printCommand,
        printError=printError,
    )
    if libs is None or libs.count == 0:
        print("No hay elementos a instalar.")
    else:
        RunCommand(
            [sys.executable, "-m", "pip", "install", *libs],
            printCommand=printCommand,
            printError=printError,
        )
        print("Deps installed.")
    print()


# Función para mostrar info el ambiente de ejecución
def ShowEnvironmentInfo():
    print("ℹ️  Environment Info:")
    print("Python Version:", sys.version)
    print("Platform:", sys.platform)
    print("Executable Path:", sys.executable)
    print("Current Working Directory:", os.getcwd())
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
    print("sys.prefix:", sys.prefix)
    print("sys.base_prefix:", sys.base_prefix)
    print()


InstallDeps(LIBS)
ShowEnvironmentInfo()
import requests


@dataclass(frozen=True)
class BoxStyle:
    TL: str
    TR: str
    BL: str
    BR: str
    H: str
    V: str

class TitleBoxLineStyle(Enum):
    SIMPLE = BoxStyle("┌", "┐", "└", "┘", "─", "│")
    DOUBLE = BoxStyle("╔", "╗", "╚", "╝", "═", "║")
    ROUNDED = BoxStyle("╭", "╮", "╰", "╯", "─", "│")
    HEAVY = BoxStyle("┏", "┓", "┗", "┛", "━", "┃")
    ASCII = BoxStyle("+", "+", "+", "+", "-", "|")
    DOUBLE_BOLD = BoxStyle("╔", "╗", "╚", "╝", "╬", "║")
    BLOCK = BoxStyle("█", "█", "█", "█", "█", "█")
    HEAVY_CROSS = BoxStyle("╒", "╕", "╘", "╛", "╪", "┃")
    METAL = BoxStyle("╞", "╡", "╘", "╛", "═", "║")


# Función para mostrar un título con recuadro
def ShowTitleBox(
    text: str,
    max_len: int = 100,
    boxLineStyle: TitleBoxLineStyle = TitleBoxLineStyle.SIMPLE,
    color: ConsoleColor = ConsoleColor.CYAN,
):
    try:

        def vislen(s: str) -> int:
            from wcwidth import wcswidth as _w

            n = _w(s)
            return n if n >= 0 else len(s)

    except Exception:

        def vislen(s: str) -> int:
            return len(s)

    pad = 1
    tlen = vislen(text)
    inner = max(max_len, tlen)
    left = (inner - tlen) // 2
    right = inner - tlen - left

    top = f"{boxLineStyle.value.TL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.TR}"
    mid = f"{boxLineStyle.value.V}{' ' * pad}{' ' * left}{text}{' ' * right}{' ' * pad}{boxLineStyle.value.V}"
    bot = f"{boxLineStyle.value.BL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.BR}"
    print(PrintColor("\n".join([top, mid, bot]), color))


# Función para descargar un archivo
def DownloadFile(uri: str, filename: str, overwrite: bool = False, timeout: int = 20):
    dest = Path(filename).resolve()
    if dest.exists() and dest.is_file() and dest.stat().st_size > 0 and not overwrite:
        print(
            f'✅ Ya existe: "{dest}". No se descarga (use overwrite=True para forzar).'
        )
        return
    if dest.parent and not dest.parent.exists():
        dest.parent.mkdir(parents=True, exist_ok=True)
    print(f'ℹ️ Descargando "{uri}" → "{dest}"')
    try:
        with requests.get(uri, stream=True, timeout=timeout) as resp:
            resp.raise_for_status()
            tmp = dest.with_suffix(dest.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in resp.iter_content(chunk_size=1024 * 64):
                    if chunk:  # filtra keep-alive chunks
                        f.write(chunk)
            tmp.replace(dest)
        print(f'✅ Archivo "{dest}" descargado exitosamente.')
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar: {e}")


# Función para descomprimir un archivo zip
def UnzipFile(filename: str, outputDir: str):
    print(f'ℹ️ Descomprimiendo "{filename}" en "{outputDir}"')
    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(outputDir)
        print(f"Descomprimido en: {os.path.abspath(outputDir)}")
    except Exception as e:
        print(f"Error: {e}")



In [25]:
# Importar librerías
import pandas as pd
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import Bunch
from sklearn.datasets import load_breast_cancer

from matplotlib.figure import Figure
from matplotlib.axes import Axes

warnings.filterwarnings("ignore")

# Configurar opciones de Pandas
pd.set_option("display.float_format", "{:.2f}".format)
pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None) 

# Función para mostrar la información del DataFrame.
def ShowDfInfo(df: pandas.DataFrame, title):
    display(f"ℹ️ INFO {title} ℹ️")
    df.info()
    display()


# Función para mostrar las n primeras filas del DataFrame.
def ShowDfHead(df: pandas.DataFrame, title: str, headQty=10):
    display(f"ℹ️ {title}: Primeros {headQty} elementos.")
    display(df.head(headQty))
    display()


# Función para mostrar las n últimas filas del DataFrame.
def ShowDfTail(df: pandas.DataFrame, title: str, tailQty=10):
    display(f"ℹ️ {title}: Últimos {tailQty} elementos.")
    display(df.tail(tailQty))
    display()


# Mostrar el tamaño del DataFrame
def ShowDfShape(df: pandas.DataFrame, title: str):
    display(f"ℹ️ {title} - Tamaño de los datos")
    display(f"{df.shape[0]} filas x {df.shape[1]} columnas")
    display()


# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowDfStats(df: pandas.DataFrame, title: str = ""):
    display(f"ℹ️ Estadística descriptiva - {title}")
    numeric_cols = df.select_dtypes(include="number")
    if not numeric_cols.empty:
        display("    🔢 Columnas numéricas".upper())
        numeric_desc = (
            numeric_cols.describe().round(2).T
        )  # Transpuesta para añadir columna
        numeric_desc["var"] = numeric_cols.var(numeric_only=True).round(2)
        display(numeric_desc.T)
    non_numeric_cols = df.select_dtypes(
        include=["boolean", "string", "category", "object"]
    )
    if not non_numeric_cols.empty:
        display("    🔡 Columnas no numéricas".upper())
        non_numeric_desc = non_numeric_cols.describe()
        display(non_numeric_desc)
    datetime_cols = df.select_dtypes(include=["datetime", "datetimetz"])
    if not datetime_cols.empty:
        display("    📅 Columnas fechas".upper())
        datetime_desc = datetime_cols.describe()
        display(datetime_desc)


# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowDfNanValues(df: pandas.DataFrame, title: str):
    display(f"ℹ️ Contador de valores Nulos - {title}")
    nulls_count = df.isnull().sum()
    nulls_df = nulls_count.reset_index()
    nulls_df.columns = ["Columna", "Cantidad_Nulos"]
    display(nulls_df)
    display()


# Tipos de correlación
class CorrelationType(Enum):
    ALL = "all"
    STRONG = "strong"
    WEAK = "weak"


# Muestra las correlaciones completas, débiles y fuertes.
def ShowDfCorrelation(
    df: pandas.DataFrame,
    title: str,
    fig: Figure,
    ax: Axes,
    level: CorrelationType = CorrelationType.ALL,
    umbral: float = 0.6,  # |r| >= umbral => fuerte; |r| <= umbral => débil
    showTable: bool = False,
    annotate: bool = True,
):
    display(f"ℹ️ {(title).upper()} - Matriz de Correlación, Type: {level.name}")
    corr = df.select_dtypes(include=["number"]).corr().copy()
    if level == CorrelationType.STRONG:
        corr = corr.where(np.abs(corr) >= umbral)
    elif level == CorrelationType.WEAK:
        corr = corr.where(np.abs(corr) <= umbral)
        np.fill_diagonal(corr.values, 1)
    elif level != CorrelationType.ALL:
        raise ValueError(f"Invalid level: {level}")
    cax = ax.matshow(corr, vmin=-1, vmax=1)

    cols = corr.columns
    ax.set_xticks(range(len(cols)))
    ax.set_yticks(range(len(cols)))
    ax.set_xticklabels(cols, rotation=90, ha="left")
    ax.set_yticklabels(cols)

    fig.colorbar(cax)

    if annotate:
        for (i, j), value in np.ndenumerate(corr.values):
            if not np.isnan(value):
                ax.text(j, i, f"{value:+.2f}", ha="center", va="center")

    if level == CorrelationType.ALL:
        titulo = "Matriz de correlación completa"
    else:
        titulo = f"Matriz de correlación ({level.name}, umbral={umbral})"

    total_elementos = corr.size
    total_nodiagonal = corr.size - corr.shape[0]
    total_nan = corr.isna().sum().sum()
    total_validos = total_elementos - total_nan - corr.shape[0]
    titulo = f"{titulo}, Total Matriz: {total_nodiagonal}, Total válidos: {total_validos}({((total_validos*100)/total_nodiagonal):.2f}%)"

    ax.set_title(titulo, pad=20)
    ax.grid(False)
    plt.tight_layout()
    plt.show()
    if showTable:
        display(corr)
    return corr


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from pandas import DataFrame
from pandas import Series

# Para almacenar los datos del dataset
@dataclass
class Dataset:
    X: DataFrame
    y: DataFrame

# Carga el dataset
def LoadDataset() -> Dataset:
    bc = cast(Bunch, load_breast_cancer(as_frame=True))
    df: DataFrame = bc.frame.copy()
    TARGET_NAME = "target"
    X = df.drop(columns=[TARGET_NAME])
    y = df[[TARGET_NAME]]
    y.columns = ["Diagnosis"]
    return Dataset(X, y)

# Para almacenar los datos de split del dataset.
@dataclass
class DatasetSplit:
    Train: Dataset
    Test: Dataset

# Realiza el split del Dataset, en Train y test utilizando el ratio.
def SplitDataset(
    data: Dataset, trainRatio: float = 0.8, randomState: int = 42
) -> DatasetSplit:
    y_strat = data.y.iloc[:, 0]
    XTrain, XTest, yTrain, yTest = train_test_split(
        data.X,
        data.y,
        train_size=trainRatio,
        random_state=randomState,
        stratify=y_strat,
    )
    return DatasetSplit(
        Train=Dataset(X=XTrain.reset_index(drop=True), y=yTrain.reset_index(drop=True)),
        Test=Dataset(X=XTest.reset_index(drop=True), y=yTest.reset_index(drop=True)),
    )

# Para almacenar los datos del dataset aplicado el escalador.
class ScaledDatasetSplit(DatasetSplit):
    pass

# Escala el split usando StandardScaler y retorna el split escalado.
def ScaleDatasetSplit(
    split: DatasetSplit, withMean: bool = True, withStd: bool = True
) -> ScaledDatasetSplit:
    scaler = StandardScaler(with_mean=withMean, with_std=withStd)
    XTrain = scaler.fit_transform(split.Train.X)
    XTest = scaler.transform(split.Test.X)
    XTrainScaled = split.Train.X.copy()
    XTestScaled = split.Test.X.copy()
    XTrainScaled.loc[:, :] = XTrain
    XTestScaled.loc[:, :] = XTest
    TrainScaled = Dataset(X=XTrainScaled, y=split.Train.y.copy())
    TestScaled = Dataset(X=XTestScaled, y=split.Test.y.copy())
    return ScaledDatasetSplit(Train=TrainScaled, Test=TestScaled)

# Muestra el head de cada componente del split.
def ShowDatasetSplitHead(split: DatasetSplit, title: str, headQty: int = 5):
    ShowDfHead(split.Train.X, f"{title} - X Train", headQty)
    ShowDfHead(split.Train.y, f"{title} - y Train", headQty)
    ShowDfHead(split.Test.X, f"{title} - X Test", headQty)
    ShowDfHead(split.Test.y, f"{title} - y Test", headQty)

# Almacena los datos del split aplicado PCA.
class PcaDatasetSplit(DatasetSplit):
    pass

# Aplica PCA al split escalado y retorna el split con PCA aplicado.
def ApplyPCA(
    scaledSplit: ScaledDatasetSplit,
    explainedVarianceRatioSum: float = 0.95,
    randomState: int = 42,
) -> PcaDatasetSplit:
    def GetPCNames(n: int) -> list[str]:
        pcs: list[str] = []
        for i in range(1, n + 1):
            pcs.append(f"PC{i}")
        return pcs

    pca = PCA(n_components=explainedVarianceRatioSum, random_state=randomState)
    XTrainPCA = pca.fit_transform(scaledSplit.Train.X)
    XTestPCA = pca.transform(scaledSplit.Test.X)
    XTrainPcaDf = pandas.DataFrame(XTrainPCA, columns=GetPCNames(XTrainPCA.shape[1]), index=scaledSplit.Train.X.index)
    XTestPcaDf = pandas.DataFrame(XTestPCA, columns=GetPCNames(XTestPCA.shape[1]), index=scaledSplit.Test.X.index)
    return PcaDatasetSplit(Dataset(X= XTrainPcaDf, y = scaledSplit.Train.y),
                           Dataset(X= XTestPcaDf, y = scaledSplit.Test.y))

# Muestra la información del Dataset
def ShowDatasetInfo(data: Dataset, title):
    tAux = title
    title = f"{tAux} - Caracteristicas - X"
    ShowDfInfo(data.X, title)
    ShowDfShape(data.X, title)
    ShowDfStats(data.X, title)
    ShowDfNanValues(data.X, title)
    ShowDfHead(data.X, title)
    ShowDfTail(data.X, title)
    title = f"{tAux} - Características - y"
    ShowDfInfo(data.y, title)
    ShowDfShape(data.y, title)
    ShowDfStats(data.y, title)
    ShowDfNanValues(data.y, title)
    ShowDfHead(data.y, title)
    ShowDfTail(data.y, title)






## 1️⃣ Exploración inicial del dataset

In [26]:

ShowTitleBox("EXPLORACIÓN INICIAL", color=ConsoleColor.MAGENTA, boxLineStyle= TitleBoxLineStyle.DOUBLE)
# ---

ShowTitleBox("CARGANDO EL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
data: Dataset = LoadDataset()
title = "Data original"
ShowDatasetInfo(data, title)

ShowTitleBox("HACIENDO SPLIT AL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
split: DatasetSplit = SplitDataset(data)
title = "Split del Dataset"
ShowDatasetSplitHead(split, title)

ShowTitleBox("HACIENDO ESCALADO AL SPLIT", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
scaled = ScaleDatasetSplit(split)
title = "Split escalado"
ShowDatasetSplitHead(split, title)


ShowTitleBox("APLICANDO PCA AL SPLIT ESCALADO", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
pcaSplit = ApplyPCA(scaled)
title = "Split con PCA"
ShowDatasetSplitHead(pcaSplit, title)

╔══════════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                         EXPLORACIÓN INICIAL                                          ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════╝
┌──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                         CARGANDO EL DATASET                                          │
└──────────────────────────────────────────────────────────────────────────────────────────────────────┘


'ℹ️ INFO Data original - Caracteristicas - X ℹ️'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

'ℹ️ Data original - Caracteristicas - X - Tamaño de los datos'

'569 filas x 30 columnas'

...

,Diagnosis
0,1
1,0
2,1
3,1
4,1


'ℹ️ Split con PCA - X Test: Primeros 5 elementos.'

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,6.42,-2.04,0.63,1.72,0.41,-0.10,1.46,0.52,0.23,0.37
1,-4.60,-0.69,0.16,-0.48,0.58,0.33,0.20,-0.01,0.16,-0.07
2,2.82,3.22,-0.75,1.24,-1.67,-0.64,0.22,0.11,-0.05,0.92
3,-0.73,-2.13,0.29,-0.11,-0.02,-0.88,-0.77,-0.06,0.79,-0.34
4,6.52,-3.78,2.51,-1.17,0.09,-2.39,-0.46,-0.42,-1.00,0.34


'ℹ️ Split con PCA - y Test: Primeros 5 elementos.'

,Diagnosis
0,0
1,1
2,0
3,1
4,0
